In [1]:
import pandas as pd
import numpy as np

In [218]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [219]:

print('Shape:',train.shape)
train = train[train.columns[[True]+list((train.var()!=0))]]
print('After dropping 0 var:',train.shape)

Shape: (4459, 4993)
After dropping 0 var: (4459, 4737)


In [220]:
print('Shape:',train.shape)
corr_matrix = train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
train=train.drop(to_drop, axis=1)
print('After dropping corr>0.95:',train.shape)

Shape: (4459, 4737)
After dropping corr>0.95: (4459, 4475)


In [221]:
corrs = dict()
for i in range(train.shape[1]-2):
    corrs[train.columns[2+i]] = np.corrcoef(train['target'],train[train.columns[2+i]])[0,1]
    
s = [k for k in corrs if abs(corrs[k])<0.1]

In [222]:
print('Shape:',train.shape)
train=train.drop(s, axis=1)
print('After dropping corr<0.1:',train.shape)

Shape: (4459, 4475)
After dropping corr<0.1: (4459, 392)


In [223]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X=train[train.columns[2:]]
y=train['target']

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

scal = StandardScaler()
x_train = scal.fit_transform(x_train)
x_test = scal.transform(x_test)


## Test Improvement

In [234]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mean_squared_logarithmic_error',
              optimizer=Adam(lr=0.1,decay=0.0001))




checkp = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=400,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Train on 3567 samples, validate on 892 samples
Epoch 1/400
3567/3567 [==============================] - 1s - loss: 81.0570 - val_loss: 33.5023
Epoch 2/400
3567/3567 [==============================] - 0s - loss: 28.2942 - val_loss: 20.7357
Epoch 3/400
3567/3567 [==============================] - 0s - loss: 19.5748 - val_loss: 15.9605
Epoch 4/400
3567/3567 [==============================] - 0s - loss: 15.9069 - val_loss: 13.4747
Epoch 5/400
3567/3567 [==============================] - 0s - loss: 13.7860 - val_loss: 11.9015
Epoch 6/400
3567/3567 [==============================] - 0s - loss: 12.4032 - val_loss: 10.7735
Epoch 7/400
3567/3567 [==============================] - 0s - loss: 11.2466 - val_loss: 9.8881
Epoch 8/400
3567/3567 [==============================] - 0s - loss: 10.4019 - val_loss: 9.1617
Epoch 9/400
3567/3567 [==============================] - 0s - loss: 9.7293 - val_loss: 8.5456
Epoch 10/400
3567/3567 [==============================] - 0s - loss: 9.0868 - val_loss: 8.015

## Actual Training

In [240]:
x_train=train[train.columns[2:]]
y_train=train['target']

x_test=test[x_train.columns]

scal = StandardScaler()
x_train = scal.fit_transform(x_train)
x_test = scal.transform(x_test)


model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mean_squared_logarithmic_error',
              optimizer=Adam(lr=0.1))

checkp = ModelCheckpoint(filepath='weights_final.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=400,
        batch_size=400,
        callbacks=cbs)

Epoch 1/400
4459/4459 [==============================] - 1s - loss: 71.5525      
Epoch 2/400
4459/4459 [==============================] - 0s - loss: 22.9996     
Epoch 3/400
4459/4459 [==============================] - 0s - loss: 16.1211     
Epoch 4/400
4459/4459 [==============================] - 0s - loss: 13.1305     
Epoch 5/400
4459/4459 [==============================] - 0s - loss: 11.3797     

In [249]:
predictions=pd.DataFrame({'ID':test['ID'],'target':model.predict(x_test, verbose=1).flatten()})
print(predictions.head())
predictions.to_csv('pred.csv',index=False)

47968/49342 [============================>.] - ETA: 0s